In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')

In [2]:
from indox import IndoxRetrievalAugmentation
from indox.llms import MistralQA
from indox.embeddings import HuggingFaceEmbedding
from indox.data_loader_splitter import UnstructuredLoadAndSplit

In [3]:
Indox = IndoxRetrievalAugmentation()

In [4]:
# Using Mistral from HuggingFace
mistral_qa = MistralQA(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed_hf = HuggingFaceEmbedding(model="multi-qa-mpnet-base-cos-v1")

In [5]:
file_path = "sample.txt"

In [6]:
loader_splitter = UnstructuredLoadAndSplit(file_path=file_path,max_chunk_size=400)

In [7]:
docs = loader_splitter.load_and_chunk()

In [8]:
docs

[Document(page_content="The wife of a rich man fell sick, and as she felt that her end\n\nwas drawing near, she called her only daughter to her bedside and\n\nsaid, dear child, be good and pious, and then the\n\ngood God will always protect you, and I will look down on you\n\nfrom heaven and be near you. Thereupon she closed her eyes and\n\ndeparted. Every day the maiden went out to her mother's grave,", metadata={'filename': 'sample.txt', 'filetype': 'text/plain', 'last_modified': '2024-05-30T13:53:09'}),
 Document(page_content='and wept, and she remained pious and good. When winter came\n\nthe snow spread a white sheet over the grave, and by the time the\n\nspring sun had drawn it off again, the man had taken another wife.\n\nThe woman had brought with her into the house two daughters,\n\nwho were beautiful and fair of face, but vile and black of heart.\n\nNow began a bad time for the poor step-child. Is the stupid goose', metadata={'filename': 'sample.txt', 'filetype': 'text/plain',

In [9]:
from indox.vector_stores import ChromaVectorStore
db = ChromaVectorStore(embedding=embed_hf,collection_name="sample")

In [10]:
Indox.connect_to_vectorstore(db)

In [11]:
Indox.store_in_vectorstore(docs)

In [12]:
query = "How cinderella reach her happy ending?"

In [13]:
retriever = Indox.QuestionAnswer(vector_database=db,llm=mistral_qa,top_k=5)

In [14]:
retriever.invoke(query=query)

"Cinderella reached her happy ending by continuing to attend the royal festival each day and using the help of her magical bird friend to transform into her beautiful dress. She eventually caught the prince's attention, they fell in love, and he discovered her true identity. The royal wedding followed, and Cinderella lived happily ever after with her prince."

In [15]:
retriever.context

['by the hearth in the cinders. And as on that account she always\n\nlooked dusty and dirty, they called her cinderella.\n\nIt happened that the father was once going to the fair, and he\n\nasked his two step-daughters what he should bring back for them.\n\nBeautiful dresses, said one, pearls and jewels, said the second.\n\nAnd you, cinderella, said he, what will you have. Father',
 'cinderella expressed a wish, the bird threw down to her what she\n\nhad wished for.\n\nIt happened, however, that the king gave orders for a festival\n\nwhich was to last three days, and to which all the beautiful young\n\ngirls in the country were invited, in order that his son might choose\n\nhimself a bride. When the two step-sisters heard that they too were',
 'know where she was gone. He waited until her father came, and\n\nsaid to him, the unknown maiden has escaped from me, and I\n\nbelieve she has climbed up the pear-tree. The father thought,\n\ncan it be cinderella. And had an axe brought and cut 

## AgenticRag

In [16]:
agent = Indox.AgenticRag(llm=mistral_qa,vector_database=db,top_k=5)

In [17]:
agent.run(query)

0
                yes
1
                Although the document mentions Cinderella, it is not describing or discussing how she reached her happy ending. Instead it is discussing the events leading up to the point where she attains her goal.
2
                Score: yes
3
               Yes.
4
                no.


"The classic version of Cinderella's story concludes with her receiving a glass slipper that fits her perfectly, allowing her to identify herself to the prince and live happily ever after in the palace as his wife. This version often includes elements such as Fairy Godmother, a pumpkin carriage, and magic transformations, but the main idea is that Cinderella's kind and beautiful nature eventually leads to her finding true love and a wealthy, happy life."